In [13]:

from lib.search_index_manager import SearchIndexManager

from lib.common import MONGODB_URI
from lib.common import PROJECT_ID
from lib.common import PRIVATE_KEY
from lib.common import PUBLIC_KEY
from lib.common import CLUSTER_NAME

import pymongo

m = SearchIndexManager ( PUBLIC_KEY, PRIVATE_KEY, PROJECT_ID, CLUSTER_NAME)
client = pymongo.MongoClient( MONGODB_URI )

In [6]:
## dynamic Index Creation 

In [7]:


## index Description 

#     "collectionName": "restaurants",
#     "database": "sample_restaurants",
#     "mappings": {"dynamic": True},
#     "name": "defau

indexDescriptor = { 
    "collectionName" : "movies",
    "database" : "sample_mflix",
    "mappings" : { "dynamic" : True }, 
    "name" : "default"
}

res = m.create_index( indexDescriptor )

res

400

In [8]:
m.get_indexes("sample_mflix", "movies")

[{'collectionName': 'movies',
  'database': 'sample_mflix',
  'indexID': '63f729f1c7a22d523fbbd5eb',
  'mappings': {'dynamic': True},
  'name': 'default',
  'status': 'STEADY',
  'synonyms': []},
 {'collectionName': 'movies',
  'database': 'sample_mflix',
  'indexID': '63f72b595ecb28071122c228',
  'mappings': {'dynamic': False,
   'fields': {'plot': {'foldDiacritics': True,
     'maxGrams': 15,
     'minGrams': 2,
     'tokenization': 'edgeGram',
     'type': 'autocomplete'},
    'title': {'foldDiacritics': True,
     'maxGrams': 15,
     'minGrams': 2,
     'tokenization': 'edgeGram',
     'type': 'autocomplete'}}},
  'name': 'autocompelete1',
  'status': 'STEADY',
  'synonyms': []}]

## Auto complete index creation 

In [9]:
indexDesc={
  "name": "autocompelete1",
  "collectionName" : "movies",
  "database" : "sample_mflix",
  "mappings": {
    "dynamic": False,
    "fields": {
      "title": {
        "type": "autocomplete",
        "tokenization": "edgeGram",
        "foldDiacritics": True,
        "minGrams": 2,
        "maxGrams": 15
      },
      "plot": {
        "type": "autocomplete",
        "tokenization": "edgeGram",
        "foldDiacritics": True,
        "minGrams": 2,
        "maxGrams": 15
      }
    }
  }
}


res = m.create_index( indexDesc )

In [10]:
m.get_indexes("sample_mflix", "movies")

[{'collectionName': 'movies',
  'database': 'sample_mflix',
  'indexID': '63f729f1c7a22d523fbbd5eb',
  'mappings': {'dynamic': True},
  'name': 'default',
  'status': 'STEADY',
  'synonyms': []},
 {'collectionName': 'movies',
  'database': 'sample_mflix',
  'indexID': '63f72b595ecb28071122c228',
  'mappings': {'dynamic': False,
   'fields': {'plot': {'foldDiacritics': True,
     'maxGrams': 15,
     'minGrams': 2,
     'tokenization': 'edgeGram',
     'type': 'autocomplete'},
    'title': {'foldDiacritics': True,
     'maxGrams': 15,
     'minGrams': 2,
     'tokenization': 'edgeGram',
     'type': 'autocomplete'}}},
  'name': 'autocompelete1',
  'status': 'STEADY',
  'synonyms': []}]

In [24]:
docs = [ {
  "_id": 0,
  "name": "Springfield High",
  "mascot": "Pumas",
  "teachers": [{
    "first": "Jane",
    "last": "Smith",
    "classes": [{
      "subject": "art of science",
      "grade": "12th"
    },
    {
      "subject": "applied science and practical science",
      "grade": "9th"
    },
    {
      "subject": "remedial math",
      "grade": "12th"
    },
    {
      "subject": "science",
      "grade": "10th"
    }]
  },
  {
    "first": "Bob",
    "last": "Green",
    "classes": [{
      "subject": "science of art",
      "grade": "11th"
    },
    {
      "subject": "art art art",
      "grade": "10th"
    }]
  }]
}, 
{
  "_id": 1,
  "name": "Evergreen High",
  "mascot": "Jaguars",
  "teachers": [{
    "first": "Jane",
    "last": "Earwhacker",
    "classes": [{
      "subject": "art",
      "grade": "9th"
    },
    {
      "subject": "science",
      "grade": "12th"
    }]
  },
  {
    "first": "John",
    "last": "Smith",
    "classes": [{
      "subject": "math",
      "grade": "12th"
    },
    {
      "subject": "art",
      "grade": "10th"
    }]
  }]
},
{
  "_id": 2,
  "name": "Lincoln High",
  "mascot": "Sharks",
  "teachers": [{
    "first": "Jane",
    "last": "Smith",
    "classes": [{
      "subject": "science",
      "grade": "9th"
    },
    {
      "subject": "math",
      "grade": "12th"
    }]
  },
  {
    "first": "John",
    "last": "Redman",
    "classes": [{
      "subject": "art",
      "grade": "12th"
    }]
  }]
}
]


coll = client["sample_mflix"]["embedded_object"]
coll.drop()
coll.insert_many ( docs )

indexDesc={
  "name": "default",
  "collectionName" : "embedded_object",
  "database" : "sample_mflix",
  "mappings": {
    "fields": {
      "teachers": {
        "type": "embeddedDocuments",
        "dynamic": True,
        "fields": {
          "classes": {
            "type": "embeddedDocuments",
            "dynamic": True
          }
        }
      }
    }
  }
}
m.delete_index( indexDesc )
m.create_index(indexDesc)


200

In [25]:
indexDesc={
  "name": "facetindex",
  "collectionName" : "movies",
  "database" : "sample_mflix",
  "mappings": {
    "dynamic": False,
    "fields": {
      "genres": {
        "type": "stringFacet"
      },
      "year": {
        "type": "numberFacet"
      },
      "released": {
        "type": "date"
      }
    }
  }
}
m.create_index(indexDesc)

200

In [28]:
docs= [
    { "item" : 1 , "tags" : [ {"k" : "Name", "v" : "SomeName"}, { "k": "Owner" , "v" : "SomeOwner" }] } ,
    { "item" : 2 , "tags" : [ {"k" : "Name2", "v" : "SomeName"}, { "k": "Owner2" , "v" : "SomeOwner" }] } 
]


coll = client["sample_mflix"]["tag_example"]
coll.drop()
coll.insert_many ( docs )

In [ ]:
indexDesc={
  "name": "autocompelete1",
  "collectionName" : "tag_example",
  "database" : "sample_mflix",
  "mappings": {
    "dynamic": False,
    "fields": {
      "tags.k": {
        "type": "autocomplete",
        "tokenization": "edgeGram",
        "foldDiacritics": True,
        "minGrams": 2,
        "maxGrams": 15
      },
      "tags.v": {
        "type": "autocomplete",
        "tokenization": "edgeGram",
        "foldDiacritics": True,
        "minGrams": 2,
        "maxGrams": 15
      }
    }
  }
}

m.delete_index( indexDesc )
m.create_index(indexDesc)